In [ ]:
import pulp
import math
import random
import json
import time
import os

DATA_PATH = 'data/mwoz/origin/data.json'

# MultiWoZ 2.0

In [ ]:
def load_data(data_path=DATA_PATH):
    with open(data_path) as f:
        data = json.load(f)

    # Remove dialogs in police & hospital
    data2 = {}
    for idx, dialog in data.items():
        if dialog['goal']['police'] or dialog['goal']['hospital']:
            continue
        data2[idx] = dialog
    data = data2

    return data

In [4]:
def solve_wdic(data, weighted=True, method='lp_rounding', k=None):
    # Extract dialogue IDs
    dialogues = list(data.keys())
    n = len(dialogues)
    
    # Initialize sets for intents covered by each dialogue (S_i), utterance counts (utterance_counts_i), and costs (c_i)
    S = [set() for _ in range(n)]          # Subsets S_i: intents covered by dialogue i
    utterance_counts = [0] * n             # Number of utterances (turns) in dialogue i
    c = [0] * n                            # Costs c_i: utterance counts (weighted) or 1 (unweighted)
    intent_set = set()                     # Universe of all intents (I)

    # Step 1: Extract intents, utterance counts, and set costs
    for i, did in enumerate(dialogues):
        dialog = data[did]
        if 'log' not in dialog:
            continue  # Skip invalid dialogue structures
        utterance_counts[i] = len(dialog['log'])  # Utterance count is the number of turns
        c[i] = utterance_counts[i] if weighted else 1  # Set cost based on weighted/unweighted
        for turn in dialog['log']:
            if 'dialog_act' in turn:
                # Construct intents as domain-act-slot combinations
                for da, slot_list in turn['dialog_act'].items():
                    for slot_val in slot_list:
                        slot = slot_val[0]
                        intent = f"{da}-{slot}"
                        S[i].add(intent)
                        intent_set.add(intent)

    # Step 2: Create the universe of intents
    intent_list = list(intent_set)
    m = len(intent_list)
    print(f"Sample intents (first 5): {intent_list[:5]}")
    print(f"Number of dialogues: {n}, Number of unique intents: {m}")
    
    if m == 0:
        print("No intents found. Exiting.")
        return [], 0.0, 0, 0, set(), 0.0, 0.0

    # Step 3: Map intents to indices for efficient processing
    intent_to_idx = {intent: j for j, intent in enumerate(intent_list)}
    
    # Step 4: Build list of dialogues covering each intent
    covering_dialogs = [[] for _ in range(m)]
    for i in range(n):
        for intent in S[i]:
            j = intent_to_idx[intent]
            covering_dialogs[j].append(i)

    # Step 5: Optimization phase (timed)
    start_time = time.time()
    D_prime_idx = []  # Indices of selected dialogues
    U = set(range(m))  # Residual uncovered intents

    if method == 'greedy':
        # Pure greedy: No LP, directly iterate to select best ratio
        while U and (k is None or len(D_prime_idx) < k):
            best_i = None
            best_ratio = 0.0
            for i in range(n):
                if i not in D_prime_idx:
                    intersect_count = sum(1 for intent in S[i] if intent_to_idx[intent] in U)
                    if intersect_count > 0:
                        ratio = intersect_count / c[i]
                        if ratio > best_ratio:
                            best_ratio = ratio
                            best_i = i
            if best_i is None:
                print("Warning: Full coverage not achievable with greedy.")
                break
            D_prime_idx.append(best_i)
            for intent in S[best_i]:
                j = intent_to_idx[intent]
                U.discard(j)

    elif method in ['lp_rounding', 'ilp']:
        # Formulate the (I)LP
        prob = pulp.LpProblem("WDIC_LP", pulp.LpMinimize)
        cat = pulp.LpBinary if method == 'ilp' else pulp.LpContinuous
        x = pulp.LpVariable.dicts("x", range(n), lowBound=0, upBound=1, cat=cat)
        
        # Objective: Minimize sum c_i * x_i
        prob += pulp.lpSum(c[i] * x[i] for i in range(n)), "Total_Cost"
        
        # Constraints: Ensure each intent is covered
        for j in range(m):
            if covering_dialogs[j]:
                prob += pulp.lpSum(x[i] for i in covering_dialogs[j]) >= 1, f"Cover_Intent_{j}"

        # Solve the (I)LP
        status = prob.solve(pulp.PULP_CBC_CMD(msg=0))
        if pulp.LpStatus[status] != "Optimal":
            print(f"No optimal solution found for {method}.")
            return [], 0.0, 0, 0, set(), 0.0, 0.0

        if method == 'ilp':
            # Exact ILP: Select where x_i == 1
            D_prime_idx = [i for i in range(n) if x[i].value() >= 0.5]  # Binary, but use >=0.5 for safety
            # Update U for coverage check (though ILP should cover all if feasible)
            for i in D_prime_idx:
                for intent in S[i]:
                    j = intent_to_idx[intent]
                    U.discard(j)
            if U:
                print("Warning: ILP did not achieve full coverage (infeasible?).")

        elif method == 'lp_rounding':
            x_star = [x[i].value() for i in range(n)]
            alpha = math.log(m) + 1  # Improved multiplier (natural log +1)
            best_D_prime_idx = []
            best_cost = float('inf')
            best_covered = 0
            for trial in range(10):  # Multiple trials for better results
                D_prime_idx_temp = []
                U_temp = set(range(m))
                for i in range(n):
                    if random.uniform(0, 1) <= min(1, alpha * x_star[i]):
                        D_prime_idx_temp.append(i)
                        for intent in S[i]:
                            j = intent_to_idx[intent]
                            U_temp.discard(j)
                # Greedy refinement
                while U_temp and (k is None or len(D_prime_idx_temp) < k):
                    best_i = None
                    best_ratio = 0.0
                    for i in range(n):
                        if i not in D_prime_idx_temp:
                            intersect_count = sum(1 for intent in S[i] if intent_to_idx[intent] in U_temp)
                            if intersect_count > 0:
                                ratio = intersect_count / c[i]
                                if ratio > best_ratio:
                                    best_ratio = ratio
                                    best_i = i
                    if best_i is None:
                        break
                    D_prime_idx_temp.append(best_i)
                    for intent in S[best_i]:
                        j = intent_to_idx[intent]
                        U_temp.discard(j)
                # Compute temp results
                temp_cost = sum(utterance_counts[i] for i in D_prime_idx_temp)
                temp_covered = m - len(U_temp)
                if temp_covered == m and temp_cost < best_cost:
                    best_cost = temp_cost
                    best_D_prime_idx = D_prime_idx_temp[:]
                    best_covered = temp_covered
            D_prime_idx = best_D_prime_idx
            # Update U based on best
            U = set(range(m))
            for i in D_prime_idx:
                for intent in S[i]:
                    j = intent_to_idx[intent]
                    U.discard(j)
            if U:
                print("Warning: Best LP-rounding trial did not achieve full coverage.")

    else:
        raise ValueError(f"Unsupported method: {method}")

    time_taken = time.time() - start_time

    # Step 6: Compute results
    selected_dialogues = [dialogues[i] for i in D_prime_idx]
    num_dialogues = len(D_prime_idx)
    total_utterances = sum(utterance_counts[i] for i in D_prime_idx)
    if weighted:
        objective_score = total_utterances  # For weighted, objective is total utterances
    else:
        objective_score = num_dialogues  # Report number of dialogues as objective score for unweighted
    covered_intents = set()
    for i in D_prime_idx:
        covered_intents.update(S[i])
    
    # Calculate coverage percentage
    coverage_percentage = (len(covered_intents) / m * 100) if m > 0 else 0.0
    
    # Identify uncovered intents (if any)
    uncovered_intents = intent_set - covered_intents

    # Step 7: Output results
    mode = "Weighted" if weighted else "Unweighted"
    print(f"{mode} Mode - Method: {method}")
    print(f"Selected dialogues: {selected_dialogues}")
    print(f"Objective score: {objective_score}")
    print(f"Number of selected dialogues: {num_dialogues}")
    print(f"Total utterances in selected dialogues: {total_utterances}")
    print(f"Number of intents covered: {len(covered_intents)} out of {m}")
    print(f"Coverage percentage: {coverage_percentage:.2f}%")
    print(f"Time taken: {time_taken:.2f} seconds")
    if uncovered_intents:
        print(f"Uncovered intents: {list(uncovered_intents)[:5]} (showing up to 5)")
    else:
        print("All intents covered.")

    # with open("selected_dialogues_MultiWoZ.txt", "w") as f:
    #     for did in selected_dialogues:
    #         f.write(f"{did}\n")
    
    return selected_dialogues, objective_score, num_dialogues, total_utterances, covered_intents, coverage_percentage, time_taken

# Example usage: Run both weighted and unweighted versions for different methods
data = load_data()

methods = ['greedy', 'lp_rounding', 'ilp']  # 'ilp' may be slow; comment out if needed
# methods = ['ilp'] 
for method in methods:
    print(f"\nRunning Weighted Version with {method}:")
    weighted_results = solve_wdic(data, weighted=True, method=method)
    
    print(f"\nRunning Unweighted Version with {method}:")
    unweighted_results = solve_wdic(data, weighted=False, method=method)


Running Weighted Version with greedy:
Sample intents (first 5): ['Attraction-Recommend-Addr', 'Attraction-Inform-Area', 'Restaurant-Inform-Choice', 'Restaurant-Select-Food', 'Hospital-Inform-Phone']
Number of dialogues: 9906, Number of unique intents: 264
Weighted Mode - Method: greedy
Selected dialogues: ['PMUL4075.json', 'PMUL3915.json', 'SNG1327.json', 'SNG0085.json', 'SNG0465.json', 'PMUL4188.json', 'SSNG0351.json', 'SNG0373.json', 'PMUL1369.json', 'SSNG0003.json', 'PMUL1337.json', 'WOZ20264.json', 'SNG01238.json', 'SNG0941.json', 'MUL0181.json', 'PMUL4352.json', 'SSNG0050.json', 'SNG1155.json', 'PMUL4984.json', 'PMUL3982.json', 'PMUL2782.json', 'SNG0974.json', 'SSNG0105.json', 'SNG0330.json', 'PMUL4261.json', 'SSNG0356.json', 'SSNG0120.json', 'SNG1372.json', 'SNG01908.json', 'WOZ20322.json', 'MUL2135.json', 'PMUL2635.json', 'SNG1193.json', 'MUL0530.json', 'PMUL4265.json', 'PMUL4172.json', 'SNG0443.json', 'SNG0285.json', 'PMUL1160.json', 'SSNG0172.json', 'PMUL4769.json', 'PMUL0480

# SGD

In [5]:
DATA_PATH = 'data/sgd/origin'

def load_dialogs(data_dir=DATA_PATH):
    
    def load_dialogs_split(split):
        dialogs = []
        data_sub_dir = os.path.join(data_dir, split)
        print(f'Loading dialogs from "{data_sub_dir}"...')
        for name in os.listdir(data_sub_dir):
            if name.startswith('dialogues_'):
                with open(os.path.join(data_sub_dir, name)) as f:
                    dialogs += json.load(f)
        for dialog in dialogs:
            dialog['dialogue_id'] = split + '_' + dialog['dialogue_id']
        return dialogs

    dialogs = []
    dialogs += load_dialogs_split('train')
    dialogs += load_dialogs_split('dev')
    dialogs += load_dialogs_split('test')

    dialogs = {d['dialogue_id']: d for d in dialogs}

    print(f'Loading completed. {len(dialogs)} dialogs loaded.')
    return dialogs


In [6]:
def solve_wdic(dialogs, weighted=True, method='lp_rounding', k=None):
    # Extract dialogue data from dictionary values
    dialog_list = list(dialogs.values())
    dialogues = [d['dialogue_id'] for d in dialog_list if isinstance(d, dict) and 'dialogue_id' in d]
    n = len(dialogues)
    if n == 0:
        print("No valid dialogues found.")
        return [], 0.0, 0, 0, set(), 0.0, 0.0

    # Initialize sets for intents (S_i), utterance counts, costs (c_i), and intent universe
    S = [set() for _ in range(n)]
    utterance_counts = [0] * n
    c = [0] * n
    intent_set = set()

    # Step 1: Extract intents and costs
    for i, dialog in enumerate(dialog_list):
        if not isinstance(dialog, dict) or 'turns' not in dialog:
            continue
        utterance_counts[i] = len(dialog['turns'])
        c[i] = utterance_counts[i] if weighted else 1
        for turn in dialog['turns']:
            for frame in turn.get('frames', []):
                service = frame.get('service', '')
                for action in frame.get('actions', []):
                    act = action.get('act', '')
                    slot = action.get('slot', '')
                    if act and slot:
                        intent_str = f"{service}-{act}-{slot}"
                        S[i].add(intent_str)
                        intent_set.add(intent_str)

    # Step 2: Create intent universe
    intent_list = list(intent_set)
    m = len(intent_list)
    print(f"Sample intents (first 5): {intent_list[:5]}")
    print(f"Number of dialogues: {n}, Number of unique intents: {m}")
    
    if m == 0:
        print("No intents found. Exiting.")
        return [], 0.0, 0, 0, set(), 0.0, 0.0

    # Step 3: Map intents to indices
    intent_to_idx = {intent: j for j, intent in enumerate(intent_list)}
    
    # Step 4: Build list of dialogues covering each intent
    covering_dialogs = [[] for _ in range(m)]
    for i in range(n):
        for intent in S[i]:
            j = intent_to_idx[intent]
            covering_dialogs[j].append(i)

    # Step 5: Optimization phase (timed)
    start_time = time.time()
    D_prime_idx = []
    U = set(range(m))

    if method == 'greedy':
        while U and (k is None or len(D_prime_idx) < k):
            best_i = None
            best_ratio = 0.0
            for i in range(n):
                if i not in D_prime_idx:
                    intersect_count = sum(1 for intent in S[i] if intent_to_idx[intent] in U)
                    if intersect_count > 0:
                        ratio = intersect_count / c[i]
                        if ratio > best_ratio:
                            best_ratio = ratio
                            best_i = i
            if best_i is None:
                print("Warning: Full coverage not achievable with greedy.")
                break
            D_prime_idx.append(best_i)
            for intent in S[best_i]:
                j = intent_to_idx[intent]
                U.discard(j)

    elif method in ['lp_rounding', 'ilp']:
        prob = pulp.LpProblem("WDIC_LP", pulp.LpMinimize)
        cat = pulp.LpBinary if method == 'ilp' else pulp.LpContinuous
        x = pulp.LpVariable.dicts("x", range(n), lowBound=0, upBound=1, cat=cat)
        prob += pulp.lpSum(c[i] * x[i] for i in range(n)), "Total_Cost"
        for j in range(m):
            if covering_dialogs[j]:
                prob += pulp.lpSum(x[i] for i in covering_dialogs[j]) >= 1, f"Cover_Intent_{j}"
        status = prob.solve(pulp.PULP_CBC_CMD(msg=0))
        if pulp.LpStatus[status] != "Optimal":
            print(f"No optimal solution found for {method}.")
            return [], 0.0, 0, 0, set(), 0.0, 0.0
        if method == 'ilp':
            D_prime_idx = [i for i in range(n) if x[i].value() >= 0.5]
            for i in D_prime_idx:
                for intent in S[i]:
                    j = intent_to_idx[intent]
                    U.discard(j)
            if U:
                print("Warning: ILP did not achieve full coverage.")
        elif method == 'lp_rounding':
            x_star = [x[i].value() for i in range(n)]
            alpha = math.ceil(math.log(m))
            for i in range(n):
                if random.uniform(0, 1) <= min(1, alpha * x_star[i]):
                    D_prime_idx.append(i)
                    for intent in S[i]:
                        j = intent_to_idx[intent]
                        U.discard(j)
            while U and (k is None or len(D_prime_idx) < k):
                best_i = None
                best_ratio = 0.0
                for i in range(n):
                    if i not in D_prime_idx:
                        intersect_count = sum(1 for intent in S[i] if intent_to_idx[intent] in U)
                        if intersect_count > 0:
                            ratio = intersect_count / c[i]
                            if ratio > best_ratio:
                                best_ratio = ratio
                                best_i = i
                if best_i is None:
                    print("Warning: Full coverage not achievable with LP-rounding.")
                    break
                D_prime_idx.append(best_i)
                for intent in S[best_i]:
                    j = intent_to_idx[intent]
                    U.discard(j)

    time_taken = time.time() - start_time

    # Step 6: Compute results
    selected_dialogues = [dialogues[i] for i in D_prime_idx]
    num_dialogues = len(D_prime_idx)
    total_utterances = sum(utterance_counts[i] for i in D_prime_idx)
    objective_score = total_utterances
    covered_intents = set()
    for i in D_prime_idx:
        covered_intents.update(S[i])
    coverage_percentage = (len(covered_intents) / m * 100) if m > 0 else 0.0
    uncovered_intents = intent_set - covered_intents

    # Step 7: Output results
    mode = "Weighted" if weighted else "Unweighted"
    print(f"{mode} Mode - Method: {method}")
    print(f"Selected dialogues: {selected_dialogues}")
    print(f"Objective score (total utterances): {objective_score}")
    print(f"Number of selected dialogues: {num_dialogues}")
    print(f"Total utterances in selected dialogues: {total_utterances}")
    print(f"Number of intents covered: {len(covered_intents)} out of {m}")
    print(f"Coverage percentage: {coverage_percentage:.2f}%")
    print(f"Time taken: {time_taken:.2f} seconds")
    if uncovered_intents:
        print(f"Uncovered intents: {list(uncovered_intents)[:5]}")
    else:
        print("All intents covered.")
    
    return selected_dialogues, objective_score, num_dialogues, total_utterances, covered_intents, coverage_percentage, time_taken

# Run all methods for both modes

dialogs = load_dialogs()

results = []
methods = ['greedy', 'lp_rounding', 'ilp']

for method in methods:
    print(f"\nRunning Weighted Version with {method}:")
    weighted_results = solve_wdic(dialogs, weighted=True, method=method)
    results.append(('Weighted', method, weighted_results))
    
    print(f"\nRunning Unweighted Version with {method}:")
    unweighted_results = solve_wdic(dialogs, weighted=False, method=method)
    results.append(('Unweighted', method, unweighted_results))

Loading dialogs from "data/sgd/origin\train"...
Loading dialogs from "data/sgd/origin\dev"...
Loading dialogs from "data/sgd/origin\test"...
Loading completed. 768 dialogs loaded.

Running Weighted Version with greedy:
Sample intents (first 5): ['RentalCars_1-INFORM_INTENT-intent', 'Music_3-CONFIRM-device', 'Events_3-REQUEST-city', 'Restaurants_1-OFFER-restaurant_name', 'Hotels_4-CONFIRM-location']
Number of dialogues: 768, Number of unique intents: 256
Weighted Mode - Method: greedy
Selected dialogues: ['test_2_00095', 'dev_1_00040', 'test_1_00024', 'train_1_00127', 'dev_2_00083', 'test_2_00046', 'dev_2_00058', 'test_1_00105', 'dev_2_00017', 'train_2_00078', 'test_2_00000', 'dev_1_00084', 'dev_2_00124', 'test_1_00016', 'test_2_00070', 'test_2_00117', 'test_1_00039', 'dev_1_00080', 'test_2_00019', 'train_2_00026', 'train_2_00090', 'test_1_00030', 'test_1_00042', 'dev_1_00113', 'test_1_00118', 'dev_2_00045', 'dev_1_00126', 'test_2_00119', 'dev_1_00037', 'dev_2_00117', 'train_1_00026', '